<a href="https://colab.research.google.com/github/abelsare348/codes/blob/pyspark/Office/2024_09_24_2_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('spark_json').getOrCreate()

In [ ]:
anpr_df=spark.read.format("json") \
          .option("multiLine", True) \
          .option("header",True) \
          .option("inferschema",True) \
          .load("/content/20240825T0000_Axis_ANPR.json")

Code to process all files from a folder of Axis_ANPR

In [ ]:
# # Directory path
# import os

# directory_path = "/home/aniket/Downloads/JsonRawdataFiles/20240825/"

# # List all files in the directory
# files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

# # Empty DataFrame to hold concatenated data
# anpr_df = None

# # Process files in directory
# print("Files in directory:")
# for i in range(0, len(files), 1):
#     filepath = files[i]

#     # Check if "DBA data" is in the file name and "000" is not in the file name #and "000" not in filepath
#     if "Axis_ANPR" in filepath :
#         # Read the JSON file into a PySpark DataFrame
#         df = spark.read.json(filepath)

#         # Concatenate the DataFrame (using union)
#         if anpr_df is None:
#             anpr_df = df
#         else:
#             anpr_df = anpr_df.union(df)

# # Show the count of rows in the final DataFrame
# print("Total rows in concatenated DataFrame:", anpr_df.count())

In [ ]:
from pyspark.sql.functions import explode, col

# Explode the array column `Data Axis_ANPR`
df_exploded = anpr_df.withColumn("exploded_ANPR", explode(col("Data Axis_ANPR")))

# Access the nested fields in the exploded struct
anpr_df_flattened = df_exploded.select(
    col("exploded_ANPR.events.currentTimestampUs").alias("currentTimestampUs"),
    col("exploded_ANPR.events.vcount.direction.in").alias("direction_in"),
    col("exploded_ANPR.events.vcount.direction.out").alias("direction_out"),
    col("exploded_ANPR.events.vcount.direction.unk").alias("direction_unk"),
    col("exploded_ANPR.events.vcount.list.blocklist").alias("blocklist"),
    col("exploded_ANPR.events.vcount.list.allowlist").alias("allowlist"),
    col("exploded_ANPR.events.vcount.list.customlist").alias("customlist"),
    col("exploded_ANPR.events.vcount.list.nonelist").alias("nonelist"),
    col("exploded_ANPR.events.vcount.roi.roi1").alias("roi1"),
    col("exploded_ANPR.events.vcount.roi.roi2").alias("roi2"),
    col("exploded_ANPR.events.vcount.total").alias("total")
)

# Show the flattened DataFrame
anpr_df_flattened.show(truncate=False)

+------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|currentTimestampUs|direction_in|direction_out|direction_unk|blocklist|allowlist|customlist|nonelist|roi1|roi2|total|
+------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|1724536805978920  |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|1724536816287082  |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|1724536826685860  |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|1724536837306731  |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|1724536847957683  |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|1724536858725779  |0           |0            |0        

In [ ]:
from pyspark.sql.functions import col, from_unixtime, to_utc_timestamp, from_utc_timestamp
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [ ]:
df = anpr_df_flattened.withColumn("currentTimestampUs", (F.col("currentTimestampUs") / 1e6).cast(T.TimestampType()))

df = df.withColumn("currentTimestampUs", F.from_utc_timestamp(F.col("currentTimestampUs"), "+02:00"))

df.show(truncate=False)

+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|currentTimestampUs        |direction_in|direction_out|direction_unk|blocklist|allowlist|customlist|nonelist|roi1|roi2|total|
+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|2024-08-25 00:00:05.97892 |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:16.287082|0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:26.68586 |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:37.306731|0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:47.957683|0           |0            |0            |0        |0        |0         |0       |0   |0   

In [ ]:
anpr_df_final=df
anpr_df_final.show(truncate=False)

+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|currentTimestampUs        |direction_in|direction_out|direction_unk|blocklist|allowlist|customlist|nonelist|roi1|roi2|total|
+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+
|2024-08-25 00:00:05.97892 |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:16.287082|0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:26.68586 |0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:37.306731|0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |
|2024-08-25 00:00:47.957683|0           |0            |0            |0        |0        |0         |0       |0   |0   

In [ ]:
anpr_df_final.printSchema()

root
 |-- currentTimestampUs: timestamp (nullable = true)
 |-- direction_in: string (nullable = true)
 |-- direction_out: string (nullable = true)
 |-- direction_unk: string (nullable = true)
 |-- blocklist: string (nullable = true)
 |-- allowlist: string (nullable = true)
 |-- customlist: string (nullable = true)
 |-- nonelist: string (nullable = true)
 |-- roi1: string (nullable = true)
 |-- roi2: string (nullable = true)
 |-- total: string (nullable = true)



In [ ]:
DBA_df=spark.read.format("json") \
          .option("multiLine", True) \
          .option("header",True) \
          .option("inferschema",True) \
          .load("/content/20240825T0000_DBA data.json")

In [ ]:
# # Directory path
# import os

# directory_path = "/home/aniket/Downloads/JsonRawdataFiles/20240825/"

# # List all files in the directory
# files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

# # Empty DataFrame to hold concatenated data
# DBA_df = None

# # Process files in directory
# print("Files in directory:")
# for i in range(0, len(files), 1):
#     filepath = files[i]

#     # Check if "DBA data" is in the file name and "000" is not in the file name #and "000" not in filepath
#     if "DBA data" in filepath :
#         # Read the JSON file into a PySpark DataFrame
#         df = spark.read.json(filepath)

#         # Concatenate the DataFrame (using union)
#         if DBA_df is None:
#             DBA_df = df
#         else:
#             DBA_df = DBA_df.union(df)

# # Show the count of rows in the final DataFrame
# print("Total rows in concatenated DataFrame:", DBA_df.count())

In [ ]:
from pyspark.sql.functions import explode, col

# Explode the outermost array `Data DBA data`
df_exploded_1 = DBA_df.withColumn("exploded_DBA_data", explode(col("Data DBA data")))

# Explode the inner array inside `exploded_DBA_data`
df_exploded_2 = df_exploded_1.withColumn("exploded_element", explode(col("exploded_DBA_data")))

# Select the fields from the exploded structure
df_flattened = df_exploded_2.select(
    col("exploded_element.id").alias("id"),
    col("exploded_element.resultType").alias("resultType"),
    col("exploded_element.timestamp").alias("timestamp"),
    col("exploded_element.locations").alias("locations"),
    col("exploded_element.values").alias("values")
)

# To further explode the `locations` array
df_locations_exploded = df_flattened.withColumn("exploded_locations", explode(col("locations")))

# Select the individual x, y, z values from the exploded locations
DBA_df_final = df_locations_exploded.select(
    col("id"),
    col("resultType"),
    col("timestamp"),
    col("exploded_locations.x").alias("location_x"),
    col("exploded_locations.y").alias("location_y"),
    col("exploded_locations.z").alias("location_z"),
    col("values").alias("values")
)

# Show the final flattened DataFrame
DBA_df_final.show(truncate=False)

+---+----------+-----------------------------------+-----------------+-----------------+-----------------+--------------------+
|id |resultType|timestamp                          |location_x       |location_y       |location_z       |values              |
+---+----------+-----------------------------------+-----------------+-----------------+-----------------+--------------------+
|1  |Event     |2024-08-25T00:00:43.536273587+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[46.830772399902344]|
|1  |Event     |2024-08-25T00:00:43.536273535+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[46.830772399902344]|
|1  |Event     |2024-08-25T00:00:51.063551509+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[48.511775970458984]|
|1  |Event     |2024-08-25T00:00:51.175898729+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[48.94055938720703] |
|1  |Event     |2024-08-25T00:00:51.288246159+02:00|4320424.244834278|565871.5884135148|4642458.98572150

In [ ]:
DBA_df_final= DBA_df_final.withColumn("timestamp", F.from_utc_timestamp(F.col("timestamp"), "+02:00"))
DBA_df_final.show(truncate=False)

+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|id |resultType|timestamp                 |location_x       |location_y       |location_z       |values              |
+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|1  |Event     |2024-08-25 00:00:43.536273|4320424.244834278|565871.5884135148|4642458.985721508|[46.830772399902344]|
|1  |Event     |2024-08-25 00:00:43.536273|4320424.244834278|565871.5884135148|4642458.985721508|[46.830772399902344]|
|1  |Event     |2024-08-25 00:00:51.063551|4320424.244834278|565871.5884135148|4642458.985721508|[48.511775970458984]|
|1  |Event     |2024-08-25 00:00:51.175898|4320424.244834278|565871.5884135148|4642458.985721508|[48.94055938720703] |
|1  |Event     |2024-08-25 00:00:51.288246|4320424.244834278|565871.5884135148|4642458.985721508|[48.128929138183594]|
|1  |Event     |2024-08-25 00:00:51.512941|43204

In [ ]:
DBA_df_final.select(F.max('timestamp'),F.min('timestamp')).show(truncate=False)

+--------------------------+--------------------------+
|max(timestamp)            |min(timestamp)            |
+--------------------------+--------------------------+
|2024-08-25 00:00:57.467354|2024-08-25 00:00:43.536273|
+--------------------------+--------------------------+



In [ ]:
anpr_df_final.select(F.max('currentTimestampUs'),F.min('currentTimestampUs')).show(truncate=False)

+--------------------------+-------------------------+
|max(currentTimestampUs)   |min(currentTimestampUs)  |
+--------------------------+-------------------------+
|2024-08-25 00:00:58.725779|2024-08-25 00:00:05.97892|
+--------------------------+-------------------------+



In [ ]:
from pyspark.sql.functions import col, abs, unix_timestamp,expr

# Convert the timestamps to unix timestamps (seconds since epoch) for comparison
df_a = anpr_df_final
df_d = DBA_df_final

# Perform a cross join between the two DataFrames (this is needed since we're doing an asof join)
cross_joined_df = df_a.join(df_d, df_a["currentTimestampUs"] >= df_d["timestamp"], "left_outer")

# Calculate the absolute difference between the timestamps
cross_joined_df = cross_joined_df.withColumn("time_diff",
                                             abs(col("currentTimestampUs") - col("timestamp")))

# Filter out rows where the time difference is greater than 3 hours (10800 seconds)
filtered_df = cross_joined_df.filter(expr("time_diff <= interval 7200 seconds"))

# Use windowing to get the nearest match within the 3-hour tolerance
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Window partitioned by events.currentTimestampUs and ordered by the time_diff
window_spec = Window.partitionBy("currentTimestampUs").orderBy(col("time_diff"))

# Add a row number and filter to keep only the first row (the nearest match)
final_df = filtered_df.withColumn("rn", row_number().over(window_spec)).filter(col("rn") == 1)

# Drop the auxiliary columns if no longer needed
final_df = final_df.drop("time_diff", "rn")

In [ ]:
final_df.show(truncate=False)

+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|currentTimestampUs        |direction_in|direction_out|direction_unk|blocklist|allowlist|customlist|nonelist|roi1|roi2|total|id |resultType|timestamp                 |location_x       |location_y       |location_z       |values              |
+--------------------------+------------+-------------+-------------+---------+---------+----------+--------+----+----+-----+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|2024-08-25 00:00:47.957683|0           |0            |0            |0        |0        |0         |0       |0   |0   |0    |1  |Event     |2024-08-25 00:00:43.536273|4320424.244834278|565871.5884135148|4642458.985721508|[46.830772399902344]|
|2024-08-25 00:00:58.725779|

<h1> Third DF Start </h1>

In [ ]:
DBC_df=spark.read.format("json") \
          .option("multiLine", True) \
          .option("header",True) \
          .option("inferschema",True) \
          .load("/content/20240825T0000_DBC data.json")

In [ ]:
# # Directory path
# import os

# directory_path = "/home/aniket/Downloads/JsonRawdataFiles/20240825/"

# # List all files in the directory
# files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

# # Empty DataFrame to hold concatenated data
# DBC_df = None

# # Process files in directory
# print("Files in directory:")
# for i in range(0, len(files), 1):
#     filepath = files[i]

#     # Check if "DBA data" is in the file name and "000" is not in the file name #and "000" not in filepath
#     if "DBC data" in filepath :
#         # Read the JSON file into a PySpark DataFrame
#         df = spark.read.json(filepath)

#         # Concatenate the DataFrame (using union)
#         if DBC_df is None:
#             DBC_df = df
#         else:
#             DBC_df = DBC_df.union(df)

# # Show the count of rows in the final DataFrame
# print("Total rows in concatenated DataFrame:", DBC_df.count())

In [ ]:
from pyspark.sql.functions import explode, col

# First explode the outer array `Data DBC data`
df_exploded_1 = DBC_df.withColumn("exploded_DBC_data", explode(col("Data DBC data")))

# Second explode the inner array within `Data DBC data`
df_exploded_2 = df_exploded_1.withColumn("exploded_element", explode(col("exploded_DBC_data")))

# Select the fields from the exploded struct
df_flattened = df_exploded_2.select(
    col("exploded_element.id").alias("id"),
    col("exploded_element.resultType").alias("resultType"),
    col("exploded_element.timestamp").alias("timestamp"),
    col("exploded_element.locations").alias("locations"),
    col("exploded_element.values").alias("values")
)

# Explode the `locations` array to access `x`, `y`, and `z`
df_locations_exploded = df_flattened.withColumn("exploded_locations", explode(col("locations")))

# Select the individual x, y, z values from the exploded locations
DBC_df_final = df_locations_exploded.select(
    col("id"),
    col("resultType"),
    col("timestamp"),
    col("exploded_locations.x").alias("location_x"),
    col("exploded_locations.y").alias("location_y"),
    col("exploded_locations.z").alias("location_z"),
    col("values").alias("values")
)

# Show the final flattened DataFrame
DBC_df_final.show(truncate=False)

+---+----------+-----------------------------------+-----------------+-----------------+-----------------+--------------------+
|id |resultType|timestamp                          |location_x       |location_y       |location_z       |values              |
+---+----------+-----------------------------------+-----------------+-----------------+-----------------+--------------------+
|2  |Event     |2024-08-24T23:59:34.966892963+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[51.28324508666992] |
|2  |Event     |2024-08-24T23:59:35.079240601+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[51.395042419433594]|
|2  |Event     |2024-08-24T23:59:35.191587873+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[50.98021697998047] |
|2  |Event     |2024-08-24T23:59:35.303935353+02:00|4320424.244834278|565871.5884135148|4642458.985721508|[51.33732223510742] |
|2  |Event     |2024-08-24T23:59:35.416282729+02:00|4320424.244834278|565871.5884135148|4642458.98572150

In [ ]:
DBC_df_final= DBC_df_final.withColumn("timestamp", F.from_utc_timestamp(F.col("timestamp"), "+02:00"))
DBC_df_final.show(truncate=False)

+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|id |resultType|timestamp                 |location_x       |location_y       |location_z       |values              |
+---+----------+--------------------------+-----------------+-----------------+-----------------+--------------------+
|2  |Event     |2024-08-24 23:59:34.966892|4320424.244834278|565871.5884135148|4642458.985721508|[51.28324508666992] |
|2  |Event     |2024-08-24 23:59:35.07924 |4320424.244834278|565871.5884135148|4642458.985721508|[51.395042419433594]|
|2  |Event     |2024-08-24 23:59:35.191587|4320424.244834278|565871.5884135148|4642458.985721508|[50.98021697998047] |
|2  |Event     |2024-08-24 23:59:35.303935|4320424.244834278|565871.5884135148|4642458.985721508|[51.33732223510742] |
|2  |Event     |2024-08-24 23:59:35.416282|4320424.244834278|565871.5884135148|4642458.985721508|[51.0670280456543]  |
|2  |Event     |2024-08-24 23:59:35.52863 |43204

In [ ]:
from pyspark.sql import functions as F
DBC_df_final.select(F.max('timestamp'),F.min('timestamp')).show(truncate=False)

+--------------------------+--------------------------+
|max(timestamp)            |min(timestamp)            |
+--------------------------+--------------------------+
|2024-08-25 00:00:58.890422|2024-08-24 23:59:34.966892|
+--------------------------+--------------------------+



In [ ]:
from pyspark.sql.functions import col, abs, unix_timestamp,expr

# Convert the timestamps to unix timestamps (seconds since epoch) for comparison
df_a = anpr_df_final
df_c = DBC_df_final

# Perform a cross join between the two DataFrames (this is needed since we're doing an asof join)
cross_joined_df = df_a.join(df_c, df_a["currentTimestampUs"] >= df_c["timestamp"], "left_outer")

# Calculate the absolute difference between the timestamps
cross_joined_df = cross_joined_df.withColumn("time_diff",
                                             abs(col("currentTimestampUs") - col("timestamp")))

# Filter out rows where the time difference is greater than 3 hours (10800 seconds)
filtered_df = cross_joined_df.filter(expr("time_diff <= interval 10800 seconds"))

# Use windowing to get the nearest match within the 3-hour tolerance
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Window partitioned by events.currentTimestampUs and ordered by the time_diff
window_spec = Window.partitionBy("currentTimestampUs").orderBy(col("time_diff"))

# Add a row number and filter to keep only the first row (the nearest match)
final_df_c = filtered_df.withColumn("rn", row_number().over(window_spec))#.filter(col("rn") == 1)

# Drop the auxiliary columns if no longer needed
#final_df_c = final_df.drop("time_diff", "rn")

In [ ]:
final_df_c.select("currentTimestampUs").groupBy("currentTimestampUs").count().show(truncate=False)

+--------------------------+-----+
|currentTimestampUs        |count|
+--------------------------+-----+
|2024-08-25 00:00:05.97892 |546  |
|2024-08-25 00:00:26.68586 |1085 |
|2024-08-25 00:00:37.306731|1358 |
|2024-08-25 00:00:58.725779|1648 |
|2024-08-25 00:00:47.957683|1550 |
|2024-08-25 00:00:16.287082|811  |
+--------------------------+-----+



In [ ]:
df_d.select(F.max('timestamp'),F.min('timestamp')).show(truncate=False)

+--------------------------+--------------------------+
|max(timestamp)            |min(timestamp)            |
+--------------------------+--------------------------+
|2024-08-25 00:00:57.467354|2024-08-25 00:00:43.536273|
+--------------------------+--------------------------+

